<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center> Автор материала: Тетерников Илья (@tetelias)

# <center>mlxtend</center>

Данная библиотека содержит модули расширения и вспомогательные инструменты для программных библиотек 
Python, предназначенных для анализа данных и машинного обучения.

В данном материале будет описаны:

* процесс установки;
* методы отбора признаков;
* прочие методы, доступные в этой библиотеке;

### <center>Установка</center>

Происходит стандартным для библиотеки Python образом:
    
    pip install mlxtend
    
Или через Anaconda:
    
    conda install -c conda-forge mlxtend

### <center>Отбор признаков методом полного перебора</center>

Сначала мы рассмотрим обертку для полного перебора всех возможных комбинаций признаков. Для этого воспользуемся алгоритмом ExhaustiveFeatureSelector, которому необходимо передать следующие параметры:
* Модель; 
* Минимальное количество признаков(`min_features=`);
* Максимальное количество признаков(`max_features=`);
* Метрика оценки(`scoring=`);
* Параметр кросс-валидации;

В качестве модели алгоритм принимает любую реализацию классификации или регрессии из scikit-learn. 

Среди метрик доступны {"accuracy", "f1", "precision", "recall", "roc_auc"} для классификации и {'mean_absolute_error', 'mean_squared_error', 'median_absolute_error', 'r2'} для регрессии.

Рассмотрим простой пример на основе стандартного набора данных __Ирис__. Свойства `best_idx_` и `best_score_` метода ExhaustiveFeatureSelector позволяют получить список индексов в списке признаков и результат метрики для лучшего набора:

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as efs
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X = iris.data
y = iris.target

knn = KNeighborsClassifier(n_neighbors=3)

fs1 = efs(knn, 
          min_features=1,
          max_features=4,
          scoring='accuracy',
          print_progress=True,
          cv=5)

fs1 = fs1.fit(X, y)

print('Best accuracy score: %.2f' % fs1.best_score_)
print('Best subset:', fs1.best_idx_)

С помощью свойства **subsets_** мы можем увидеть подробности каждого шага:

In [ ]:
fs1.subsets_

Данный метод может также применяться в процессе подбора параметров моделей с помощью GridSearchCV. Для этого необходимо применение метода make_pipeline. Для извлечения лучшего набора признаков нужно указать **refit=True** в GridSearchCV:

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as efs
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

logit = LogisticRegression(multi_class='multinomial', 
                           solver='lbfgs', 
                           random_state=123)

fs1 = efs(estimator=logit, 
          min_features=2,
          max_features=3,
          scoring='accuracy',
          print_progress=False,
          clone_estimator=False,
          cv=5,
          n_jobs=1)

pipe = make_pipeline(fs1, logit)

param_grid = {'exhaustivefeatureselector__estimator__C': [0.1, 1.0, 10.0]}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5, 
                  verbose=1, 
                  refit=True)

# run gridearch
gs = gs.fit(X_train, y_train)

Лучшие параметры GridSearchCV выводятся стандартным

In [ ]:
gs.best_params_

А вот индексы лучших признаков отыскать не так просто:

In [ ]:
gs.best_estimator_.steps[0][1].best_idx_

### <center>Отбор признаков методом последовательного перебора</center>

Полный перебор прост в исполнении, но количество вариантов растет пропорционально 2 в степени количества признаков. Для того, чтобы избежать такого массового перебора есть группа методов последовательного отбора признаков. Метод Sequential Forward Selection начинает с 0 признаков и выбирает тот, что максимально увеличивает заданную пользователем метрику. Затем к отобранному добавленяетсяием еще один и т.д.. Метод Sequential Backward Selection наоборот начинает с полного набора и отбрасывает по одному признаки менее всего положительно влияющие на заданную метрику. Есть еще надстройки над этими методами: Sequential Forward loating Selection и Sequential Backward Floating Selection. Они по сравнению с базовыми делают проверку, не улучшат ли уже отброшенные до этого признаки показатель метрики, если их все-таки добавить на текущем этапе работы алгоритма.

Необходимые параметры:
* Модель;
* Количество признаков, которое мы хотим получить на выходе, задаваемое через k_features;
* Направление прохождения алгоритма: от нулевого(`forward=True`) или полного(`forward=False`) набора признаков;
* Пытаться ли вернуть ранее отброшенные признаки: да(`floating=True`) или нет(`floating=False`);
* Метрика оценки(`scoring=`);
* Параметр кросс-валидации.

Но к ним добавляется еще и кол-во признаков, которое мы хотим получить на выходе

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X = iris.data
y = iris.target
knn = KNeighborsClassifier(n_neighbors=4)

sfs1 = sfs(knn, 
           k_features=3, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)

Как и в предыдущий раз свойство **subsets_** позволяет увидеть подробности каждого шага:

In [ ]:
sfs1.subsets_

Индексы лучшего набора признаков можно получить, используя свойство **`k_feature_idx_`**.

In [ ]:
sfs1.k_feature_idx_

Так же, как и в случае с полным перебором, данный метод можно использовать вместе с GridSearchCV. Но надо не забыть  указать в GridSearchCV **refit=True**:

In [ ]:
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

knn = KNeighborsClassifier(n_neighbors=2)

sfs1 = sfs(estimator=knn, 
           k_features=3,
           forward=True, 
           floating=False, 
           scoring='accuracy',
           cv=5)

pipe = Pipeline([('sfs', sfs1), 
                 ('knn', knn)])

param_grid = [
  {'sfs__k_features': [1, 2, 3, 4],
   'sfs__estimator__n_neighbors': [1, 2, 3, 4]}
  ]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5,                   
                  verbose=1,
                  refit=True)

# run gridearch
gs = gs.fit(X_train, y_train)

И опять получение лучшего набора признаков не выглядит элегантно:

In [ ]:
gs.best_estimator_.steps[0][1].k_feature_idx_

### <center>Другие интересные методы библиотеки</center>

Библиотека также содержит единственную на Python реализацию алгоритма Априори для построения правил ассоциативности: 
очень простого, но обладающего исключительной интерпретируемостью метода исследования данных. Помимо [небольшого пояснения от автора](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) рекомендую к прочтению вот [эту статью](http://pbpython.com/market-basket-analysis.html).



И напоследок: библиотека также имеет средства работы с текстом, включая возможность обрабатывать смайлики. Можно пытаться делать свой шаблон через библиотеку __re__ или просто использовать

In [ ]:
from mlxtend.text import tokenizer_emoticons

tokenizer_emoticons('</a>This :) is :( a test ;-)!')

Можно извлекать смайлики не только отдельно, но и вместе с текстом:

In [ ]:
from mlxtend.text import tokenizer_words_and_emoticons

tokenizer_words_and_emoticons('</a>This :) is :( a test :-)!')

# Ссылки
* [Документация](http://rasbt.github.io/mlxtend/) mlxtend
* [Репозитарий на Github](https://github.com/rasbt/mlxtend)